In [1]:
from mpi4py import MPI
from petsc4py.PETSc import ScalarType #type: ignore
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot, cpp
from dolfinx.fem.petsc import LinearProblem
from ufl import ds, dx, grad, inner
from scipy.interpolate import interp2d
from tqdm import tqdm

In [2]:
# Load heat source values
heat = np.load("/home/shengbiaolu/snap/dolfinx-main/python/demo/Params/sample_f_64_new.npy")
heat.shape

# Load thermal conductivity
thermal_conductivity = np.load("/home/shengbiaolu/snap/dolfinx-main/python/demo/Params/sample_theta_64_new.npy")
thermal_conductivity.shape

(100, 64, 64)

In [3]:
def interp_f_theta(f, theta):
    x = np.linspace(-1,1,64)
    y = np.linspace(-1,1,64)
    x, y = np.meshgrid(x,y)

    heat_interp = interp2d(x, y, f)
    conductivity_interp = interp2d(x, y, theta)

    heat_f = lambda x: heat_interp(x[0], x[1])[0]
    conductivity_theta = lambda x: conductivity_interp(x[0], x[1])[0]

    return heat_f, conductivity_theta

In [4]:
# Create a Mesh and the element is the triangle
msh = mesh.create_rectangle(
    comm = MPI.COMM_WORLD,
    points = ((-1.0, -1.0),(1.0, 1.0)),
    n=(32, 32),
    cell_type=mesh.CellType.quadrilateral
)
V = fem.functionspace(msh, ("Lagrange", 2))

In [5]:
# Define the Neumann Boundary Conditions

facets = mesh.locate_entities_boundary(
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], -1.0) | np.isclose(x[0], 1.0)
)

dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)

# Define the Dirichlet Boundary Conditions
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [6]:
# Define the variational problem 
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
u_array = np.zeros((10000, 4225))

for i in tqdm(range(10000)):
    if i == 0:
        heat_f, conductivity_theta = interp_f_theta(heat[i//100], thermal_conductivity[i%100])
        
        theta = fem.Function(V) # heat conductivity
        theta.interpolate(conductivity_theta)

        f = fem.Function(V) # Heat source
        f.interpolate(heat_f)

        g = 1
        a = inner(grad(u), theta * grad(v)) * dx
        L = inner(f, v) * dx + inner(g,theta * v) * ds

    # Linear Problem
    
        problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type":"preonly", "pc_type":"lu"})

    else:
        heat_f, conductivity_theta = interp_f_theta(heat[i//100], thermal_conductivity[i%100])
        f.interpolate(heat_f)
        theta.interpolate(conductivity_theta)
    uh = problem.solve()


    u_array[i] = uh.vector.array

100%|██████████| 10000/10000 [8:44:29<00:00,  3.15s/it] 


"T_element_input resolution_degree"

In [7]:
np.save("T_32_64_2", u_array)